In [6]:
import pandas as pd
import os
import numpy as np
import psycopg2
# from sqlalchemy import create_engine, URL, MetaData
# import scipy.io

# test_file = '/app/res/test.csv'

# file = '../../res/Combined_Flights_2021.parquet'
# file = './res/Combined_Flights_2021.parquet'
test_file = '/app/raw/Flights_2018_1.csv'
# csv_file = pd.read_csv(file_2022_6, engine='pyarrow')
# print(list(csv_file.columns))
# for line in list(csv_file.columns):
#     print(line)
sample = pd.read_csv(test_file, engine='pyarrow')
print("I'm loaded")

I'm loaded


In [9]:
# sample = csv_file.sample(n = 100000)
# sample.to_csv("../../res/test.csv", index=False)
print(len(sample.index))

457688


In [12]:
#get all the flights in the first file that either depart from or arrive to boston
bos_sample = sample.query('Origin == "BOS" or Dest == "BOS"')
#print the number of flights from or to boston 
print(len(bos_sample.index))

18149


In [ ]:
#what is the average delay time for the flights from boston

In [5]:
bos = sample.query('Origin == "BOS" or Dest == "BOS"').to_numpy()
# sample['Origin']
# sample['OriginAirportSeqID']
print(bos[0:2])

[[2018 1 1 5 5 datetime.date(2018, 1, 5) 'B6' 'B6' 20409 'B6' 797 '' nan
  '' nan 'B6' 20409 'B6' 'N663JB' 797 10721 1072102 30721 'BOS'
  'Boston, MA' 'MA' 25 'Massachusetts' 13 10397 1039707 30397 'ATL'
  'Atlanta, GA' 'GA' 13 'Georgia' 34 1607 1902.0 175.0 175.0 1.0 11.0
  '1600-1659' 29.0 1931.0 2150.0 8.0 1907 2158.0 171.0 171.0 1.0 11.0
  '1900-1959' 0.0 '' 0.0 180.0 176.0 139.0 1.0 946.0 4 171.0 0.0 0.0 0.0
  0.0 nan nan nan 0 nan nan nan nan '' nan nan nan nan nan nan '' '' nan
  nan nan nan nan nan '' None None None None None None None None None
  None None None None None None None None None None None None None None
  None 'N' None]
 [2018 1 1 10 3 datetime.date(2018, 1, 10) 'B6' 'B6' 20409 'B6' 504 ''
  nan '' nan 'B6' 20409 'B6' 'N629JB' 504 12954 1295406 32575 'LGB'
  'Long Beach, CA' 'CA' 6 'California' 91 10721 1072102 30721 'BOS'
  'Boston, MA' 'MA' 25 'Massachusetts' 13 2115 2116.0 1.0 1.0 0.0 0.0
  '2100-2159' 9.0 2125.0 517.0 5.0 529 522.0 -7.0 0.0 0.0 -1.0
  '0001-05

In [ ]:
# file = open("/app/raw/Flights_2018_1.csv")
# print(file.readline())
# print(file.readline())
# bos = file[np.where(OriginAirportID == "BOS")]
# print(bos)

# for i in file:
#     print(i)




In [ ]:
# file_csv = '../../res/Combined_Flights_2021.csv'
# for line in list(pq_file.columns):
#     print(line)
# print(csv_file.columns)

# pd.read_csv(file_csv)

In [15]:
#testing the postgresql server
# url_object = URL.create(
#     "postgresql+psycopg2",
#     username="postgres",
#     password="postgres",
#     host="flights-db",
#     port="5432",
#     database="flights"
# )

# db = create_engine(url_object)k

# with db.connect() as conn:
#     s = 
    # for row in conn.execute()

conn_string = "host='flights-db' dbname='flights' user='postgres' password='postgres'"
with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM demo")
    records = cursor.fetchall()
    print(records)

[(1, 'testing'), (2, 'today')]
